# MusicGroup Machine Learning Test

In [1]:
from utils import *

In [2]:
databaseDir = 'D:\Users\Marc\Downloads\music_group_ml_test\music_group_ml_test\music_group_ml_test_data'
jsonfiles = fetchFiles(databaseDir, '.json')
print "Number of json files fetched: " + str(len(jsonfiles))

Number of json files fetched: 2831


In [3]:
data = importData(jsonfiles)
train, test = randomTrainTest(data, percentage_train=0.3)
printDataStats(train, test)

Created training and testing sets with the following number of sounds:

	Train	Test	Total	Class

	165	383	548	hihat
	507	1180	1687	bass
	1027	2396	3423	guitar
	300	697	997	vocals
	121	280	401	tom
	69	161	230	saxophone
	234	545	779	piano
	121	280	401	snare
	121	282	403	kick

	2665	6204	8869	TOTAL


In [4]:
gmms = computeGMMS(train, n_components=2)
correct, predicted = scoreGMMS(gmms, test)
classificationReport(correct, predicted)


Classification report

             precision    recall  f1-score   support

       bass       0.93      0.90      0.92      1180
     guitar       0.92      0.97      0.95      2396
      hihat       0.88      0.77      0.82       383
       kick       0.89      0.62      0.73       282
      piano       0.96      0.97      0.96       545
  saxophone       1.00      0.52      0.69       161
      snare       0.96      0.83      0.89       280
        tom       0.80      0.82      0.81       280
     vocals       0.79      0.96      0.87       697

avg / total       0.91      0.90      0.90      6204

Confusion Matrix

[[1064   94    0   19    1    0    0    2    0]
 [   9 2323    9    0   15    0    4    0   36]
 [   6    1  296    0    0    0    3    0   77]
 [  43    8    2  174    0    0    0   55    0]
 [   3   12    0    0  526    0    0    0    4]
 [   0   14    1    0    2   84    0    0   60]
 [   0   25   15    0    4    0  232    1    3]
 [  17   26    2    2    0    0    2